# Innovation Notebook — Extensiones Futuras de la App Interoperable

Ideas y caminos de evolución técnica para la aplicación móvil que permite pagos transfronterizos instantáneos entre turistas y pequeños comercios, usando Open Payments e Interledger.




## 1. Propósito de este notebook

El MVP actual cumple con la función central: **permitir pagos instantáneos, sin fricción, entre turistas y pequeños comercios**, en cualquier moneda.
Este notebook documenta **ideas de evolución futura**, mejoras que combinan interoperabilidad, analítica y valor agregado tanto para viajeros como para negocios locales.

No es parte del producto final, sino una visión de hacia dónde puede crecer el ecosistema.

## 2. Enfoques de evolución

| Enfoque                                | Descripción                                                                                              | Valor agregado                                                      |
| -------------------------------------- | -------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------- |
| **Capa de inteligencia financiera** | Analizar datos de pagos para entender hábitos de turistas y flujos por país, moneda y categoría.         | Insight sobre comportamiento económico de turistas.             |
| **Mapa inteligente de comercios**  | Mostrar en tiempo real qué establecimientos aceptan pagos interoperables con Lana| Mejora de la experiencia del turista, visibilidad para el comercio. |
| **Ciencia de datos aplicada**       | Usar machine learning para detectar tendencias, prever demanda y sugerir estrategias a los comerciantes. | Valor agregado para ambos lados del ecosistema.                     |
| **Recompensas dinámicas**           | Diseñar un sistema de incentivos según patrones de consumo o zonas turísticas activas.                   | Fomenta la adopción de pagos digitales y fidelidad.                 |

---

## Idea 1 — Mapa inteligente de comercios

El mapa de negocio que útilicen LANA para mejor experiencia de los usuarios. A futuro buscariamos **convertirlo en un mapa vivo** que reciba datos del backend:

* Comercios con **actividad reciente** (pagos en las últimas horas).
* Filtros por **tipo de producto** o **moneda aceptada**.
* Estadísticas visuales: *“Zonas con mayor movimiento de turistas por pagos Open Payments”*.


### Ejemplo de endpoint de actividad

```js
// /api/merchant-activity
[
  { name: "Tacos El Güero", transactionsToday: 12, lastPayment: "2025-11-09T18:00Z" },
  { name: "Café Interledger", transactionsToday: 7, lastPayment: "2025-11-09T17:40Z" }
]
```

Esto permitiría **visualizar en tiempo real la economía local** impulsada por Interledger.

A continuación se comparte posiblles caminos de uso:

In [ ]:
# Backend

// merchantActivityServer.js
import express from "express";

const app = express();
const PORT = 4000;

// Datos hardcodeados de comercios simulados
const merchantActivity = [
  {
    id: 1,
    name: "Tacos El Güero",
    type: "Comida",
    currency: "MXN",
    lat: 19.4326,
    lon: -99.1332,
    transactionsToday: 12,
    lastPayment: "2025-11-09T18:00:00Z",
  },
  {
    id: 2,
    name: "Café Interledger",
    type: "Bebidas",
    currency: "USD",
    lat: 19.4330,
    lon: -99.1320,
    transactionsToday: 7,
    lastPayment: "2025-11-09T17:40:00Z",
  },
  {
    id: 3,
    name: "OpenPay Market",
    type: "Tienda",
    currency: "EUR",
    lat: 19.4315,
    lon: -99.1345,
    transactionsToday: 4,
    lastPayment: "2025-11-09T15:20:00Z",
  },
];

app.get("/api/merchant-activity", (req, res) => {
  res.json(merchantActivity);
});

app.listen(PORT, () => {
  console.log(`API demo corriendo en http://localhost:${PORT}`);
});


In [ ]:
# Frontend

// MapaDemo.tsx
import React, { useEffect, useState } from "react";
import { View, Text, ScrollView } from "react-native";
import MapView, { Marker, Callout } from "react-native-maps";

interface Merchant {
  id: number;
  name: string;
  type: string;
  currency: string;
  lat: number;
  lon: number;
  transactionsToday: number;
  lastPayment: string;
}

export default function MapaDemo() {
  const [merchants, setMerchants] = useState<Merchant[]>([]);

  useEffect(() => {
    fetch("http://localhost:4000/api/merchant-activity")
      .then((res) => res.json())
      .then(setMerchants)
      .catch((err) => console.error("Error al obtener comercios:", err));
  }, []);

  return (
    <ScrollView style={{ flex: 1, backgroundColor: "#fff" }}>
      <Text
        style={{
          fontSize: 22,
          fontWeight: "bold",
          textAlign: "center",
          marginVertical: 10,
        }}
      >
        Mapa inteligente de comercios (Demo)
      </Text>

      <MapView
        style={{ height: 400 }}
        initialRegion={{
          latitude: 19.4326,
          longitude: -99.1332,
          latitudeDelta: 0.01,
          longitudeDelta: 0.01,
        }}
      >
        {merchants.map((m) => (
          <Marker
            key={m.id}
            coordinate={{ latitude: m.lat, longitude: m.lon }}
            title={m.name}
            description={`${m.type} — ${m.currency}`}
          >
            <Callout>
              <View style={{ width: 180 }}>
                <Text style={{ fontWeight: "bold" }}>{m.name}</Text>
                <Text>Categoría: {m.type}</Text>
                <Text>Moneda: {m.currency}</Text>
                <Text>Pagos hoy: {m.transactionsToday}</Text>
                <Text>
                  Último pago:{" "}
                  {new Date(m.lastPayment).toLocaleTimeString("es-MX")}
                </Text>
              </View>
            </Callout>
          </Marker>
        ))}
      </MapView>

      <View style={{ padding: 10 }}>
        <Text style={{ textAlign: "center", color: "gray", marginTop: 10 }}>
          Datos simulados — demostración de cómo podría verse un mapa vivo con
          comercios activos usando la infraestructura de Open Payments.
        </Text>
      </View>
    </ScrollView>
  );
}


## Idea 2 — Análisis de consumo turístico

Cada transacción Open Payments genera un rastro de información que puede alimentar analítica económica:

* Origen del turista (moneda).
* Categoría del comercio (gastronomía, alojamiento, transporte).
* Volumen promedio por país.
* Horas y días con mayor actividad.

### Ejemplo de análisis con Python

```python
import pandas as pd

df = pd.DataFrame([
    {"pais":"MX", "categoria":"comida", "monto":20},
    {"pais":"US", "categoria":"transporte", "monto":35},
    {"pais":"FR", "categoria":"comida", "monto":18},
    {"pais":"JP", "categoria":"alojamiento", "monto":90},
])

gasto_por_categoria = df.groupby("categoria")["monto"].mean()
print("Gasto promedio por categoría:")
print(gasto_por_categoria)
```
Esto podría alimentar dashboards de turismo sostenible o reportes locales, mostrando cómo la infraestructura de Open Payments inyecta liquidez directamente en los pequeños comercios.

Ideas de implementación:

In [ ]:
# Endpoint
// touristAnalyticsServer.js
import express from "express";
const app = express();
const PORT = 4001;

// Datos simulados de transacciones turísticas (hardcodeados)
const transactions = [
  { id: 1, country: "MX", category: "comida", amount: 20, date: "2025-11-01T14:00:00Z" },
  { id: 2, country: "US", category: "transporte", amount: 35, date: "2025-11-02T09:30:00Z" },
  { id: 3, country: "FR", category: "comida", amount: 18, date: "2025-11-02T13:00:00Z" },
  { id: 4, country: "JP", category: "alojamiento", amount: 90, date: "2025-11-03T20:10:00Z" },
  { id: 5, country: "US", category: "comida", amount: 42, date: "2025-11-03T19:00:00Z" },
  { id: 6, country: "MX", category: "transporte", amount: 25, date: "2025-11-04T10:30:00Z" },
];

app.get("/api/tourist-consumption", (req, res) => {
  res.json(transactions);
});

app.listen(PORT, () => {
  console.log(`API demo de consumo turístico corriendo en http://localhost:${PORT}`);
});


In [ ]:
# Frontend

# tourist_analytics_demo.py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests

# Datos hardcodeados
# data = requests.get("http://localhost:4001/api/tourist-consumption").json()
data = [
    {"pais": "MX", "categoria": "comida", "monto": 20, "fecha": "2025-11-01T14:00:00Z"},
    {"pais": "US", "categoria": "transporte", "monto": 35, "fecha": "2025-11-02T09:30:00Z"},
    {"pais": "FR", "categoria": "comida", "monto": 18, "fecha": "2025-11-02T13:00:00Z"},
    {"pais": "JP", "categoria": "alojamiento", "monto": 90, "fecha": "2025-11-03T20:10:00Z"},
    {"pais": "US", "categoria": "comida", "monto": 42, "fecha": "2025-11-03T19:00:00Z"},
    {"pais": "MX", "categoria": "transporte", "monto": 25, "fecha": "2025-11-04T10:30:00Z"},
]

df = pd.DataFrame(data)
df["fecha"] = pd.to_datetime(df["fecha"])
df["hora"] = df["fecha"].dt.hour
df["dia"] = df["fecha"].dt.day_name()

# Gasto promedio por categoría
gasto_por_categoria = df.groupby("categoria")["monto"].mean().sort_values(ascending=False)
print("💰 Gasto promedio por categoría:")
print(gasto_por_categoria)

# Gasto promedio por país
gasto_por_pais = df.groupby("pais")["monto"].mean()
print("\n Gasto promedio por país:")
print(gasto_por_pais)

# Visualización simple
plt.figure(figsize=(8,4))
sns.barplot(x=gasto_por_categoria.index, y=gasto_por_categoria.values)
plt.title("Gasto promedio por categoría")
plt.ylabel("Monto promedio (USD)")
plt.xlabel("Categoría")
plt.show()

# Horas de mayor consumo
plt.figure(figsize=(8,4))
sns.histplot(df["hora"], bins=12, kde=True)
plt.title("Distribución horaria del consumo turístico")
plt.xlabel("Hora del día")
plt.ylabel("Frecuencia de pagos")
plt.show()


## Idea 3 — Ciencia de datos aplicada

### a. Segmentación de turistas

Usando clustering (`KMeans` o `DBSCAN`), se puede identificar tipos de comportamiento:

* **Ahorro** (pequeños montos, alta frecuencia)
* **Premium** (montos altos, baja frecuencia)
* **Explorador** (diversidad de comercios)

```python
from sklearn.cluster import KMeans
import pandas as pd

data = pd.DataFrame([
    {"freq":10, "avg_spend":15},
    {"freq":3, "avg_spend":80},
    {"freq":7, "avg_spend":40},
])

kmeans = KMeans(n_clusters=3, n_init="auto").fit(data)
data["segment"] = kmeans.labels_
print(data)
```

Esto permitiría personalizar promociones según el tipo de viajero.

### b. Predicción de demanda local

Entrenando un modelo de regresión (ej. `RandomForestRegressor`) sobre datos de pagos históricos:

* Día, hora, clima, feriado local nos permitiria conocer el número esperado de pagos.
* Permite sugerir a los comerciantes **cuándo prepararse para más ventas**.

El objetivo sería **optimizar la operación de pequeños comercios** con analítica simple pero útil.

In [ ]:
# Back
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt

# Datos hardcodeados de ejemplo
data = pd.DataFrame([
    {"turista":"A", "freq":10, "avg_spend":15},  # frecuentes, bajo gasto
    {"turista":"B", "freq":3, "avg_spend":80},   # premium
    {"turista":"C", "freq":7, "avg_spend":40},   # explorador
    {"turista":"D", "freq":12, "avg_spend":18},
    {"turista":"E", "freq":2, "avg_spend":100},
    {"turista":"F", "freq":6, "avg_spend":50},
])

# Entrenamiento simple del modelo
kmeans = KMeans(n_clusters=3, n_init="auto", random_state=42).fit(data[["freq", "avg_spend"]])
data["segmento"] = kmeans.labels_

# Mostrar los resultados
print("Segmentación de turistas:\n", data)

# Visualización
plt.scatter(data["freq"], data["avg_spend"], c=data["segmento"], cmap="viridis", s=80)
plt.title("Segmentación de turistas por frecuencia y gasto promedio")
plt.xlabel("Frecuencia de compras")
plt.ylabel("Gasto promedio (USD)")
plt.show()


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Datos simulados de pagos diarios
data = pd.DataFrame([
    {"dia_semana": 1, "hora": 9, "feriado": 0, "clima": 1, "transacciones": 8},
    {"dia_semana": 5, "hora": 19, "feriado": 0, "clima": 0, "transacciones": 32},
    {"dia_semana": 6, "hora": 20, "feriado": 0, "clima": 1, "transacciones": 40},
    {"dia_semana": 7, "hora": 14, "feriado": 1, "clima": 0, "transacciones": 55},
    {"dia_semana": 2, "hora": 10, "feriado": 0, "clima": 1, "transacciones": 12},
    {"dia_semana": 3, "hora": 18, "feriado": 0, "clima": 0, "transacciones": 28},
])

# División de datos
X = data[["dia_semana", "hora", "feriado", "clima"]]
y = data["transacciones"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenamiento del modelo
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Evaluación
pred = model.predict(X_test)
print(f"Error medio absoluto: {mean_absolute_error(y_test, pred):.2f}")

# Predicción hipotética
prediccion_sabado_noche = model.predict([[6, 20, 0, 1]])
print(f"🔮 Predicción de transacciones sábado noche: {prediccion_sabado_noche[0]:.1f}")



## Conclusión

Estas ideas complementan la solución actual sin modificar su núcleo:

> *Una app interoperable de pagos turísticos que puede crecer hacia una plataforma de inteligencia económica local.*

La combinación de Open Payments + analítica convierte los pagos en datos de impacto.